# UNCLASSIFIED

Transcribed from FOIA Doc ID: 6689695

https://archive.org/details/comp3321

# (U) Setup

(U) For this notebook, you will need the following files: 

- user_file.csv 

- user_file.xml

Note: I had to guess at the contents of these files since they weren't included in the source materials.

(U) Right-click each to download and "Save As," then, from your Jupyter home, navigate to the folder containing this notebook and click the "Upload" button to upload each file from your local system. 

Note: I'm including my homemade versions of these files in the github.

# (U) Introduction: It's Sad, But True 

(U) Much of computing involves reading and writing structured data. Too much, probably. Often that data is contained in files--not even a database. We've already worked with opening, closing, reading from, and writing to text files. We've also frequently used string methods. At first, it might seem that that's all we need to work with CSV, XML, and other structured data formats. 

(U) After all, what could go wrong with the following? 

In [ ]:
my_csv_file = open('user_file.csv', 'r')

In [ ]:
csv_lines = my_csv_file.read().splitlines()

In [ ]:
comma_separated_records = [line.split(',') for line in csv_lines]

In [ ]:
xml_formatter = """
  <person>
    <name>{}</name>
    <address>{}</address>
    <city>{}</city>
    <state>{}</state>
    <zip>{}</zip>
    <phone>{}</phone>
    <primary_workstation>{}</primary_workstation>
    <username>{}</username>
  </person>"""

In [ ]:
xml_records = "\n".join([xml_formatter.format(*record) for record in comma_separated_records])

In [ ]:
xml_records = "<people>" + xml_records + "</people>"

In [ ]:
with open('file.xml', 'w') as f:
    f.write(xml_records)

(U) In a rapidly-developed prototype with controlled input, this may not cause a problem. Given the way the real world works, though, someday this little snippet from a one-off script will become the long-forgotten key component of a huge, enterprise-wide project. Somebody will try to feed it data in just the wrong way at a crucial moment, and it will fail catastrophically. 

(U) When that happens, you'll wish you had used a fully-developed library that would have had a better chance against the malformed data. Thankfully, there are several-and they actually aren't any harder to get started with. 

# (U) Comma Separated Values (CSV) 

(U) The most exciting things about the `csv` module are the `DictReader` and `Dictwriter` classes. First, let's look at the plain vanilla options for reading and writing. 

In [ ]:
import csv

In [ ]:
f = open('user_file.csv')

In [ ]:
reader = csv.reader(f)

In [ ]:
header = next(reader)

In [ ]:
all_lines = [line for line in reader]

In [ ]:
all_lines.sort()

In [ ]:
g = open('user_file_sorted.csv', 'w') 

In [ ]:
writer = csv.writer(g) 

In [ ]:
writer.writerow(header) 

In [ ]:
writer.writerows(all_lines) 

In [ ]:
g.close() 

(U) CSV readers and writers have other options involving _dialects_ and _separators_. Note that the argument to `csv. reader` must be an open file (or file-like object), and the reading starts at the current cursor position. 

(U) Accessing categorical data positionally is not ideal. That is why `csv` also provides the `DictReader` and `Dictwriter` classes, which can also handle records with more or less in them than you expect. When given only a file as an argument, a `DictReader` uses the first line as the keys for the remaining lines; however, it is also possible to pass in `fieldnames` as an additional parameter.

In [ ]:
f.seek(0)

In [ ]:
d_reader = csv.DictReader(f) 

In [ ]:
records = [line for line in d_reader] 

In [ ]:
records

(U) To see the differences between reader and DictReader, look at how we might extract cities from the records in each. 

In [ ]:
# for the object from csv.reader 
cities0 = [record[2] for record in all_lines]

In [ ]:
# for the object from csv.DictReader 
cities1 = [record['city'] for record in records]

In [ ]:
cities1

In [ ]:
cities0 == cities1

(U) In a `DictWriter`, the `fieldnames` parameter is required and headers are not written by default. If you want one, add it with the `writeheader` method. If the `fieldnames` argument does not include all the fields for every dictionary passed into the `DictWriter`, the keyword argument `extrasaction` must be specified.

In [ ]:
g = open('names_only.csv', 'w')

In [ ]:
d_writer = csv.DictWriter(g, ['name', 'primary_workstation'], extrasaction='ignore')

In [ ]:
d_writer.writeheader()

In [ ]:
d_writer.writerows(records)

In [ ]:
g.close()

# (U) JavaScript Object Notation (JSON) 

(U) JSON is another structured data format. In many cases it looks very similar to nested Python `dict`s and `list`s. However, there are enough notable differences from those (e.g. only single quotation marks are allowed, boolean values have a lowercase initial letter) that it's wise to use a dedicated module to parse JSON data. Still, _serializing_ and _deserializing_ JSON data structures is relatively painless.

(U) For this section, our example will be a list of novels:

In [ ]:
import json 

In [ ]:
novel_list = []

In [ ]:
novel_list.append({'title': 'Pride and Prejudice', 'author': 'Jane Austen'}) 

In [ ]:
novel_list.append({'title': 'Crime and Punishment', 'author': 'Fyodor Dostoevsky'})

In [ ]:
novel_list.append({'title': 'The Unconsoled' , 'author': 'Kazuo Ishiguro'}) 

In [ ]:
json.dumps(novel_list) # to string 

In [ ]:
with open('novel_list.json', 'w') as f: 
    json.dump(novel_list, f) # to file

In [ ]:
the_hobbit = '{"title": "The Hobbit", "author": "J.R.R. Tolkien"}'

In [ ]:
novel_list.append(json.loads(the_hobbit)) # from string 

In [ ]:
with open('war_and_peace.json') as f: # <-- if this file existed 
    novel_list.append(json.load(f)) # from fiie 

(U) By default, the `load` and `loads` methods return Unicode strings. It's possible to use the **json** module to define custom encoders and decoders, but this is not usually required. 

# (U) Extensible Markup Language (XML) 

(U) This lesson is supposed to be simple, but XML is complicated. We'll cover only the basics of reading data from and writing data to files in a very basic XML format using the **ElementTree** API, which is just the most recent of at least three approaches to dealing with XML in the Python Standard Library. We will not discuss attributes or namespaces at all, which are very common features of XML. If you need to process lots of XML quickly, it's 
probably best to look outside the standard library (probably at a package called **lxml**). 

(U) Although there are other ways to get started, an `ElementTree` can be created from a file by initializing with the keyword argument `file`: 

In [ ]:
from xml.etree import ElementTree

In [ ]:
xml_file = open('user_file.xml')

In [ ]:
user_tree = ElementTree.ElementTree(file=xml_file)

(U) To do much of anything, it's best to pull the root element out of the `ElementTree`. Elements are iterable, so they can be expanded in list comprehensions. To see what is inside an element, the **ElementTree** module provides two class functions: `dump` (which prints to screen and returns `None`) and `tostring`. Each node has a `text` property, although in our example these are all empty except for leaf nodes. 

In [ ]:
root_elt = user_tree.getroot()

In [ ]:
users = [u for u in root_elt]

In [ ]:
print(ElementTree.tostring(users[0]))

In [ ]:
u_children = [x for x in users[1]]

In [ ]:
u_children[2].text

In [ ]:
u_children[2].text = 'north-x5-1234'

In [ ]:
ElementTree.dump(users[1])

(U) To get nested descendant elements directly, use `findall`, which returns a list of all matches, or `find`, which returns the first matched element. Note that these are the actual elements, not copies, so changes made here are visible in the whole element tree.

In [ ]:
all_usernames = root_elt.findall('user/name/username')

In [ ]:
all_usernames

In [ ]:
[n.text for n in all_usernames[1:10]]

(U) To construct an XML document: 

- make an `Element`, 
- `append` other `Element`s to it (repeating as necessary), 
- wrap it all up in an `ElementTree`, and 
- use the `ElementTree.write` method (which takes a file _name_, not a `file` object).

In [ ]:
apple = ElementTree.Element('apple')

In [ ]:
apple.attrib['color'] = 'red'

In [ ]:
apple.set('variety', 'honeycrisp')

In [ ]:
apple.text = "Tasty"

In [ ]:
ElementTree.dump(apple)

In [ ]:
fruit_basket = ElementTree.Element('basket')

In [ ]:
fruit_basket.append(apple)

In [ ]:
fruit_basket.append(ElementTree.XML('<orange color="orange" variety="navel"></orange>'))

In [ ]:
ElementTree.dump(fruit_basket)

In [ ]:
fruit_tree = ElementTree.ElementTree(fruit_basket)

In [ ]:
fruit_tree.write('fruit_basket.xml')

# (U) Bonus Material: Pickles and Shelves 

(U) At the expense of compatibility with other languages, Python also provides built-in serialization and data storage capabilities in the form of the **pickle** and **shelve** modules.

**pickle** lets you seralize objects into a byte stream that can be saved to a binary file and **shelve** expands on that by letting you assign key names to those objects in a byte steam.

## (U) Pickling

In [ ]:
import pickle

In [ ]:
pickleme = {}

In [ ]:
pickleme['Title'] = 'Python is Cool'

In [ ]:
pickleme['PageCount'] = 543

In [ ]:
pickleme['Author'] = 'PythonFanboy1994'

In [ ]:
with open('/tmp/pickledData.pick', 'wb') as p:
    p = pickle.dump(pickleme, p) 

In [ ]:
with open('/tmp/pickledData.pick', 'rb') as p:
    p = pickle.load(p) 

In [ ]:
print(p)

## (U) Shelving 
### (U) Creating a Shelve 

In [ ]:
import shelve

In [ ]:
pickleme = {}

In [ ]:
pickleme['Title'] = 'Python is Cool'

In [ ]:
pickleme['PageCount'] = 543

In [ ]:
pickleme['Author'] = 'PythonFanboy1994'

In [ ]:
db = shelve.open('/tmp/shelve.dat')

In [ ]:
db['book1'] = pickleme

In [ ]:
db.sync()

In [ ]:
pickleme['Title'] = 'Python is Cool -- The Next Phase'

In [ ]:
pickleme['PageCount'] = 123

In [ ]:
pickleme['Author'] = 'PythonFanboy1994'

In [ ]:
db['book2'] = pickleme

In [ ]:
db.sync()

In [ ]:
db.close()

### (U) Opening a Shelve 

In [ ]:
db = shelve.open('/tmp/shelve.dat')

In [ ]:
z = db.keys()

In [ ]:
a = db['book1']

In [ ]:
b = db['book2']

In [ ]:
print(a)

In [ ]:
print(b)

In [ ]:
print(z)

In [ ]:
db.close()

### (U) Modifying a Shelve 

In [ ]:
db = shelve.open('/tmp/shelve.dat')

In [ ]:
z = db.keys()

In [ ]:
a = db['book1']

In [ ]:
b = db['book2']

In [ ]:
print(a)

In [ ]:
print(b)

In [ ]:
print(z)

In [ ]:
a['PageCount'] = 544

In [ ]:
b['PageCount'] = 129

In [ ]:
db['book1'] = a

In [ ]:
db['book2'] = b

In [ ]:
db.close()

# UNCLASSIFIED

Transcribed from FOIA Doc ID: 6689695

https://archive.org/details/comp3321